In [5]:
# !pip install transformers
import transformers

### Using pre-trained transformers
_for fun and profit_

There are many toolkits that let you access pre-trained transformer models, but the most powerful and convenient by far is [`huggingface/transformers`](https://github.com/huggingface/transformers). In this week's practice, you'll learn how to download, apply and modify pre-trained transformers for a range of tasks. Buckle up, we're going in!


__Pipelines:__ if all you want is to apply a pre-trained model, you can do that in one line of code using pipeline. Huggingface/transformers has a selection of pre-configured pipelines for masked language modelling, sentiment classification, question aswering, etc. ([see full list here](https://huggingface.co/transformers/main_classes/pipelines.html))

A typical pipeline includes:
* pre-processing, e.g. tokenization, subword segmentation
* a backbone model, e.g. bert finetuned for classification
* output post-processing

Let's see it in action:

In [6]:
from transformers import pipeline

In [3]:
classifier = pipeline('sentiment-analysis', model="distilbert-base-uncased-finetuned-sst-2-english")
print(classifier("BERT is amazing!"))

[{'label': 'POSITIVE', 'score': 0.9998860359191895}]


In [4]:
import base64
data = {
    'arryn': 'As High as Honor.',
    'baratheon': 'Ours is the fury.',
    'stark': 'Winter is coming.',
    'tyrell': 'Growing strong.'
}

# YOUR CODE: predict sentiment for each noble house and create outputs dict
outputs = {house: True if classifier(desc)[0]['label'] == 'POSITIVE' else False for house, desc in data.items() }

assert sum(outputs.values()) == 3 and outputs[base64.decodestring(b'YmFyYXRoZW9u\n').decode()] == False
print("Well done!")

Well done!


<ipython-input-4-71af068e669e>:12: DeprecationWarning: decodestring() is a deprecated alias since Python 3.1, use decodebytes()
  assert sum(outputs.values()) == 3 and outputs[base64.decodestring(b'YmFyYXRoZW9u\n').decode()] == False


You can also access vanilla Masked Language Model that was trained to predict masked words. Here's how:

In [5]:
mlm_model = pipeline('fill-mask', model="bert-base-uncased")
MASK = mlm_model.tokenizer.mask_token

for hypo in mlm_model(f"Donald {MASK} is the president of the united states."):
  print(f"P={hypo['score']:.5f}", hypo['sequence'])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


P=0.99719 donald trump is the president of the united states.
P=0.00024 donald duck is the president of the united states.
P=0.00022 donald ross is the president of the united states.
P=0.00020 donald johnson is the president of the united states.
P=0.00018 donald wilson is the president of the united states.


In [6]:
# Your turn: use bert to recall what year was the Soviet Union founded in
mlm_model(f"During the existence of Soviet Union from {MASK} to 1991, it spanned across europe and asia")

[{'sequence': 'during the existence of soviet union from 1917 to 1991, it spanned across europe and asia',
  'score': 0.15198418498039246,
  'token': 4585,
  'token_str': '1917'},
 {'sequence': 'during the existence of soviet union from 1918 to 1991, it spanned across europe and asia',
  'score': 0.12480923533439636,
  'token': 4271,
  'token_str': '1918'},
 {'sequence': 'during the existence of soviet union from 1922 to 1991, it spanned across europe and asia',
  'score': 0.06078825145959854,
  'token': 4798,
  'token_str': '1922'},
 {'sequence': 'during the existence of soviet union from 1945 to 1991, it spanned across europe and asia',
  'score': 0.05101242661476135,
  'token': 3386,
  'token_str': '1945'},
 {'sequence': 'during the existence of soviet union from 1921 to 1991, it spanned across europe and asia',
  'score': 0.03918205946683884,
  'token': 4885,
  'token_str': '1921'}]

In [7]:
# 1922 seems to be the right answer

```

```

```

```


Huggingface offers hundreds of pre-trained models that specialize on different tasks. You can quickly find the model you need using [this list](https://huggingface.co/models).


In [15]:
text = """Almost two-thirds of the 1.5 million people who viewed this liveblog had Googled to discover
 the latest on the Rosetta mission. They were treated to this detailed account by the Guardian’s science editor,
 Ian Sample, and astronomy writer Stuart Clark of the moment scientists landed a robotic spacecraft on a comet 
 for the first time in history, and the delirious reaction it provoked at their headquarters in Germany.
  “We are there. We are sitting on the surface. Philae is talking to us,” said one scientist.
"""

# Task: create a pipeline for named entity recognition, use task name 'ner' and search for the right model in the list
ner_model = pipeline('token-classification', model="dslim/bert-base-NER")

named_entities = ner_model(text)

In [16]:
print('OUTPUT:', named_entities)
word_to_entity = {item['word']: item['entity'] for item in named_entities}
assert 'org' in word_to_entity.get('Guardian').lower() and 'per' in word_to_entity.get('Stuart').lower()
print("All tests passed")

OUTPUT: [{'entity': 'B-LOC', 'score': 0.7991049, 'index': 27, 'word': 'Rose', 'start': 112, 'end': 116}, {'entity': 'I-LOC', 'score': 0.9511926, 'index': 28, 'word': '##tta', 'start': 116, 'end': 119}, {'entity': 'B-ORG', 'score': 0.998223, 'index': 40, 'word': 'Guardian', 'start': 179, 'end': 187}, {'entity': 'B-PER', 'score': 0.9997613, 'index': 46, 'word': 'Ian', 'start': 207, 'end': 210}, {'entity': 'I-PER', 'score': 0.99978703, 'index': 47, 'word': 'Sam', 'start': 211, 'end': 214}, {'entity': 'I-PER', 'score': 0.999646, 'index': 48, 'word': '##ple', 'start': 214, 'end': 217}, {'entity': 'B-PER', 'score': 0.9997831, 'index': 53, 'word': 'Stuart', 'start': 240, 'end': 246}, {'entity': 'I-PER', 'score': 0.99974823, 'index': 54, 'word': 'Clark', 'start': 247, 'end': 252}, {'entity': 'B-LOC', 'score': 0.9997227, 'index': 85, 'word': 'Germany', 'start': 414, 'end': 421}, {'entity': 'B-PER', 'score': 0.9963128, 'index': 99, 'word': 'Phil', 'start': 471, 'end': 475}, {'entity': 'I-PER', '

### The building blocks of a pipeline

Huggingface also allows you to access its pipelines on a lower level. There are two main abstractions for you:
* `Tokenizer` - converts from strings to token ids and back
* `Model` - a pytorch `nn.Module` with pre-trained weights

You can use such models as part of your regular pytorch code: insert is as a layer in your model, apply it to a batch of data, backpropagate, optimize, etc.

In [31]:
import torch
from transformers import AutoTokenizer, AutoModel, pipeline

model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [32]:
lines = [
    "Luke, I am your father.",
    "Life is what happens when you're busy making other plans.",
    ]

# tokenize a batch of inputs. "pt" means [p]y[t]orch tensors
tokens_info = tokenizer(lines, padding=True, truncation=True, return_tensors="pt")

for key in tokens_info:
    print(key, tokens_info[key])

print("Detokenized:")
for i in range(2):
    print(tokenizer.decode(tokens_info['input_ids'][i]))

input_ids tensor([[ 101, 5355, 1010, 1045, 2572, 2115, 2269, 1012,  102,    0,    0,    0,
            0,    0,    0],
        [ 101, 2166, 2003, 2054, 6433, 2043, 2017, 1005, 2128, 5697, 2437, 2060,
         3488, 1012,  102]])
token_type_ids tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
Detokenized:
[CLS] luke, i am your father. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] life is what happens when you're busy making other plans. [SEP]


In [34]:
# You can now apply the model to get embeddings
with torch.no_grad():
    output = model(**tokens_info)

print(output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.3502,  0.2246, -0.2345,  ..., -0.2232,  0.1730,  0.6747],
         [-0.6097,  0.6892, -0.5512,  ..., -0.4814,  0.5322,  1.3833],
         [ 0.1842,  0.4881,  0.2193,  ..., -0.2699,  0.2246,  0.7985],
         ...,
         [-0.4413,  0.2748, -0.0391,  ..., -0.0604, -0.4358,  0.1384],
         [-0.5414,  0.4633,  0.0678,  ..., -0.1871, -0.5046,  0.2752],
         [-0.3940,  0.6180,  0.2092,  ..., -0.2345, -0.4177,  0.3341]],

        [[ 0.1622, -0.1154, -0.3894,  ..., -0.4180,  0.0138,  0.7644],
         [ 0.6471,  0.3774, -0.4082,  ...,  0.0050,  0.5559,  0.4385],
         [ 0.3351, -0.3158, -0.1178,  ...,  0.1348, -0.3143,  1.4409],
         ...,
         [ 1.2932, -0.1743, -0.5613,  ..., -0.2718, -0.1367,  0.4217],
         [ 1.0305,  0.1708, -0.2985,  ...,  0.2097, -0.4627, -0.4277],
         [ 1.0854,  0.1760, -0.0377,  ...,  0.3152, -0.5979, -0.3465]]]), pooler_output=tensor([[-0.8854, -0.4722, -0.9392,  .

### The search for similar questions.

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to **really** solve this task using context-aware embeddings.

In [25]:
# download the data:
# !wget "https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1" -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

__Main task(3 pts):__ 
* Implement a function that takes a text string and finds top-k most similar questions from `quora.txt`
* Demonstrate your function using at least 5 examples

There are no prompts this time: you will have to write everything from scratch.

In [26]:
# Download quora questions
with open("quora.txt", "r") as file:
    corpus_lines = file.read().split("\n")
    
print(len(corpus_lines))

537273


In [42]:
# Some test data
# corpus_lines = [
#     "What is neural network?",
#     "How do you prepare fish fast?",
#     "Why is death a bad thing?"
# ]

In [75]:
import numpy as np
from scipy.spatial.distance import cosine

# The main idea was to use score based on usage of named entites and contextual embeddings
# and determine closest questions based on this method

def find_tok_k_similar_questions(question_text, k=3):
    united_corpus = corpus_lines.copy()
    united_corpus.append(question_text)
    
    question_idx = len(united_corpus) - 1
    
    tokens = tokenizer(united_corpus, padding=True, truncation=True, return_tensors="pt")
    
    with torch.no_grad():
        sentence_embeddings = model(**tokens).pooler_output
        
    question_emb = sentence_embeddings[question_idx]
    corpus_len = question_idx
        
    corpus_ner = [set([entry['entity'] for entry in ner_model(line)]) for line in united_corpus]
    question_ner = set([entry['entity'] for entry in ner_model(question_text)])
    
    entity_match_score = [len(q.intersection(question_ner)) / len(mask) for q, mask in zip(corpus_ner[:corpus_len], tokens['attention_mask'])]
    cosine_score = 1 / np.array([1.001 + cosine(question_emb, phrase_emb) for phrase_emb in sentence_embeddings[:corpus_len]])

    total_score = entity_match_score + cosine_score
    
    top_k = np.argsort(-total_score)[:k]
    nearest = []
    for i in range(0, k):
        nearest.append(united_corpus[top_k[i]])
    
    return nearest
        
    print(token_embeddings)

In [76]:
find_tok_k_similar_questions("What's the main idea of transformer-based neural networks?")

['What is neural network?',
 'Why is death a bad thing?',
 'How do you prepare fish fast?']

```















```

__Bonus demo:__ transformer language models. 

`/* No points awarded for this task, but its really cool, we promise :) */`

In [1]:
import torch
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', add_prefix_space=True)
model = GPT2LMHeadModel.from_pretrained('gpt2').train(False).to(device)

text = "The Fermi paradox "
tokens = tokenizer.encode(text)
num_steps = 1024
line_length, max_length = 0, 70

 The Fermi paradox 

In [6]:
nucleus_p = 0.8

print(end=tokenizer.decode(tokens))

for i in range(num_steps):
    with torch.no_grad():
        logits = model(torch.as_tensor([tokens], device=device))[0]
    p_next = torch.softmax(logits[0, -1, :], dim=-1)

    # next_token_index = p_next.argmax() #<YOUR CODE: REPLACE THIS LINE>
    # YOUR TASK: change the code so that it performs nucleus sampling
    
    sorted_p_next, sorted_indices = torch.sort(p_next, descending=True)
    cum_prob = np.cumsum(sorted_p_next.cpu().detach().numpy())
    last_index = np.sum((cum_prob >= nucleus_p))
    
    token_probs = sorted_p_next[:last_index]
    tokens_to_sample = sorted_indices[:last_index]
    
    probs = np.array(token_probs)
    probs = probs / np.linalg.norm(probs, ord=1) # l1-norm
    next_token_index = np.random.choice(tokens_to_sample, p=probs)

    tokens.append(int(next_token_index))
    print(end=tokenizer.decode(tokens[-1]))
    line_length += len(tokenizer.decode(tokens[-1]))
    if line_length >= max_length:
        line_length = 0
        print()

 The Fermi paradox  meant that integral probabilities can be introduced as constructs, so that if either of these can be studied in Aristotle's time, then we can see how such constructs are subtypes of reality (Queueda, 1992).
Observing an imp o'phralable structure
I have described the dynamical conditions that such structure can occupy in concept orientational space prior to passing through the^NP irons of my functional hypothesis. As I have argued, the reality unfolds from the top down without normally occurring coupling nodes or radio silence. My discrete factorial focus is towards PRIMARY BURROS DISTINGENT FATE ISSUE, hence I consider the de implicating unity
 principle especially important in quantitative theory.   Briefly, I shall
 infer the evidence for the non-canonical factorial

KeyboardInterrupt: 

Transformers knowledge hub: https://huggingface.co/transformers/